Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np
#print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
#sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

In [2]:
import pylab as pl

In [3]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [4]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [5]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [6]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

print om0

0.306732450988


In [7]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [8]:
#calculate results for these parameters
results = camb.get_results(pars)

In [9]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=2.0, npoints = 200)

In [10]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [11]:
#Euclid spectroscopic galaxy survey

#Redshift bins
zlist = np.arange(0.7,1.45,0.1)
ztest = zlist[4]
Nzbins = len(zlist)

#mean number density of galaxies in each redshift bin
factor = pow(hubble,3)*1e-4
nbarlist = [17.5, 19, 18, 17, 15, 13, 12, 10]

print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4]
ztest = 1.1
Number of redshift bins = 8


In [12]:
#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
#Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)

print rcom(ztest)

3642.17535167


In [13]:
#Define the growth function in LCDM
def get_growth(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

print get_growth(ztest)

0.8877954272


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return get_growth(zz)/(1+zz)
def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

print Dgz(ztest)

0.582875695343


In [15]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial bg
def bg(zc):
    return np.sqrt(1+zc)

#stochasticity coefficient
rstoc=1.0

#fiducial OmHI Mario's fit
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)


print OmHI(ztest), bHI(ztest), bg(ztest)

0.00083035 0.9285 1.44913767462


In [16]:
#Construct P(k,z) 
def Pkz(kk,zc):    
    return pow(Dgz(zc),2)*Pkz0(kk)

In [17]:
#mean brightness temperature [mK] Mario's fit
def Tb(zc):
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

In [18]:
#Construct P_HI(k,z) [mK^2]
def PHI(kk,zc):
    return pow(Tb(zc),2)*pow(bHI(zc),2)*Pkz(kk,zc)

In [19]:
#HI intensity mapping survey noise specs (MeerKAT)
Ndishes=64
Ddish=13.5*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

Area=500.0 #deg^2
omegatot = Area*pow(pi/180,2)
ttotal = 4000*60*60*(Area/4000) #note it varies with Area

Tsyslist = [23.5*1e3,23.0*1e3,23.0*1e3,28.0*1e3,29.0*1e3,30.0*1e3,28.5*1e3,29.5*1e3,\
            31.0*1e3,33.0*1e3,34.0*1e3,35.0*1e3,37.0*1e3,38.0*1e3] #mK

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1
dfpix = 50*1e3 #Hz
midfreq = 1420.4e6 #Hz

def dzpix(zc):
    return pow(1+zc,2)*dfpix/midfreq
def sigpix(zc,Tsys):
    return Tsys/np.sqrt(dfpix*tobs(zc)) 
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

def Wsq(kk,zc):
    return np.exp(-pow(kk,2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

def Pnoise(kk,zc,Tsys):
    return pow(sigpix(zc,Tsys),2)*Vpix(zc)*pow(Wsq(kk,zc),-1.)

In [20]:
#optical galaxy survey 
def Pgg(kk,zc):
    return pow(bg(zc),2)*Pkz(kk,zc)

def Pshot(zc):
    return 1/nbar

In [21]:
def kmin(zc):
    return 2*pi/np.sqrt(pow(rcom(zc),2)*omegatot)
def kmax(zc):
    return 0.14*pow(1+zc,2/(2+nss)); #non-linear cutoff Smith et al 2003
    
#print kmin(ztest), kmax(ztest)

In [22]:
#Construct P_{HI,g}(k,z)

def PHIg(kk,zc):
    return Tb(zc)*bHI(zc)*bg(zc)*rstoc*Pkz(kk,zc)

In [23]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)


def Veff(kk,zc):
    return Vsur(zc)*(pow(PHIg(kk,zc),2)/(pow(PHIg(kk,zc),2)+(PHI(kk,zc)+Pnoise(kk,zc,Tsys))
                                         *(Pgg(kk,zc)+Pshot(zc))))

In [24]:
#Fisher matrix derivatives

def dlnP_dlnOmHIbHIr(kk,zc):
    return 1.0

In [25]:
#Fisher matrix

def dFdk(kk):
    return (1./(4*pi*pi))*pow(kk,2)*pow(dlnP_dlnOmHIbHIr(kk,zc),2)*Veff(kk,zc)

In [26]:
for i in range(0,Nzbins):
    zc = zlist[i]
    Tsys = Tsyslist[i]
    nbar = factor*nbarlist[i]
    K = np.linspace(kmin(zc), kmax(zc), 500)
    dF = dFdk(K)
    Fisher = scipy.integrate.simps(dF,K)           
    print zc, np.sqrt(1/Fisher)

0.7 0.0581261120794
0.8 0.0667479246893
0.9 0.0768953241344
1.0 0.094772732306
1.1 0.109649305197
1.2 0.125905086916
1.3 0.138680147078
1.4 0.157231968425
